In [1]:
# import system modules
import sys
import os
import gc

# import standard numerical modules
import numpy as np
import math

import matplotlib.pyplot as plt

# import machine learning modules
import tensorflow as tf
import keras.backend as K

sys.path.append('../')
import DCTR


2024-08-16 13:22:07.475123: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-08-16 13:22:07.475173: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# Process Data

In [2]:
data_dir = '../../Data'
num_events = int(5e8) # 985997 num MiNNLO events


## hvq dielptonic


In [3]:
# dileptonic hvq has 1M events per LHE

X0 = []
X0_nJets = []
X0_jet = []
for i in range(1, 21):
    X0.append(
        np.load(f'{data_dir}/POWHEG_hvq/dileptonic/converted_lhe_hvq_dileptonic_1M_{i}.npy')
    )
    X0_nJets.append(
        np.load(f'{data_dir}/POWHEG_hvq/dileptonic/jet_multiplicity_hvq_dileptonic_1M_{i}.npy')
    )
    X0_jet.append(
        np.load(f'{data_dir}/POWHEG_hvq/dileptonic/jet_4vectors_hvq_dileptonic_1M_{i}.npy')
    )

# nrm data and save to disk (only needed to run once)

nrm_array_dir = f'{data_dir}/POWHEG_hvq/showered/'
nrm_array = np.load(f'{nrm_array_dir}/norm_array_lhe_01.npy')
X0_nrm, _ = DCTR.normalize_data(X0.copy(), nrm_array)
X0_nrm = np.array(X0_nrm)
np.save(f'{data_dir}/POWHEG_hvq/dileptonic/normed_lhe_hvq_dileptonic_1M_1-20.npy', X0_nrm)

X0       = np.array(X0[num_events])
X0_nJets = np.array(X0_nJets[num_events])
X0_jet   = np.array(X0_jet[num_events])

X0_nrm = np.load(f'{data_dir}/POWHEG_hvq/dileptonic/normed_lhe_hvq_dileptonic_1M_1-20.npy')[:num_events]
X0_wgt = X0_nJets[:,1]

print(X0.shape)
print(X0_nrm.shape)
print(X0_nJets.shape)
print(X0_jet.shape)


FileNotFoundError: [Errno 2] No such file or directory: '../../Data/POWHEG_hvq/dileptonic/jet_multiplicity_hvq_dileptonic_1M_1.npy'

## MiNNLO

In [ ]:
# MiNNLO has 10k events per lhe

X1 = []
X1_nJets = []
X1_jet = []

for i in range(1, 1001):
    X1.extend(
        np.load(f'{data_dir}/MiNNLO/showered/converted_lhe_MiNNLO_10K_{i}.npy')
    )
    X1_nJets.extend(
        np.load(f'{data_dir}/MiNNLO/showered/jet_multiplicity_MiNNLO_10K_{i}.npy')
    )
    X1_jet.extend(
        np.load(f'{data_dir}/MiNNLO/showered/jet_4vectors_MiNNLO_10K_{i}.npy')
    )


# nrm data and save to disk (only needed to run once)

nrm_array_dir = f'{data_dir}/POWHEG_hvq/showered/'
nrm_array = np.load(f'{nrm_array_dir}/norm_array_lhe_01.npy')
X1_nrm, _ = DCTR.normalize_data(X1.copy(), nrm_array)
np.save(f'{data_dir}/MiNNLO/showered/normed_lhe_MiNNLO_10K_1-1000.npy', X1_nrm)


X1       = np.array(X1[:num_events])
X1_nJets = np.array(X1_nJets[:num_events])
X1_jet   = np.array(X1_jet[:num_events])

X1_nrm = np.load(f'{data_dir}/MiNNLO/showered/normed_lhe_MiNNLO_10K_1-1000.npy')[:num_events]
X1_wgt = X1_nJets[:, 1]

print(X1.shape)
print(X1_nrm.shape)
print(X1_nJets.shape)
print(X1_jet.shape)


# prepare datasets

In [ ]:
# delete energy from (normalized) testing dataset, since the neural network was trained without these parameters
X0_nrm = np.delete(X0_nrm, 5, -1) # E
X0_nrm = np.delete(X0_nrm, 4, -1) # eta

# garbage collection after deleting to clear memory asap
print(gc.collect())

print(X0_nrm.shape)
print(X0_nrm[0])


# compare datasets before reweighting

### hvq

In [ ]:
# jet multiplicity

args_multiplicity = [(X0_minPT_15_nJets[:,0], X0_wgt, 'Jet minPT 15 Multiplicity NLO (hvq) \n dileptonic decays'),
                     (X0_minPT_30_nJets[:,0], X0_wgt, 'Jet minPT 30 Multiplicity NLO (hvq) \n dileptonic decays'),
                     (X0_minPT_50_nJets[:,0], X0_wgt, 'Jet minPT 50 Multiplicity NLO (hvq) \n dileptonic decays')]


bins = np.linspace(-0.5, 10.5, 12)

DCTR.plot_ratio_cms(args_multiplicity, bins = bins, y_scale = 'log', ratio_ylim=[0.7, 1.3], part_label='Jet', arg_label='multiplicity', unit='', inv_unit='')


### MiNNLO

In [ ]:
# jet multiplicity

args_multiplicity = [(X1_minPT_15_nJets[:,0], X1_wgt, 'Jet minPT 15 Multiplicity NNLO (MiNNLO) \n dileptonic decays'),
                     (X1_minPT_30_nJets[:,0], X1_wgt, 'Jet minPT 30 Multiplicity NNLO (MiNNLO) \n dileptonic decays'),
                     (X1_minPT_50_nJets[:,0], X1_wgt, 'Jet minPT 50 Multiplicity NNLO (MiNNLO) \n dileptonic decays')]


bins = np.linspace(-0.5, 10.5, 12)

DCTR.plot_ratio_cms(args_multiplicity, bins = bins, y_scale = 'log', ratio_ylim=[0.7, 1.3], part_label='Jet', arg_label='multiplicity', unit='', inv_unit='')


## hvq vs MiNNLO

In [ ]:
# jet multiplicity

args_multiplicity = [(X0_minPT_15_nJets[:,0], X0_wgt, 'Jet minPT 15 Multiplicity NLO (hvq) \n dileptonic decays'),
                     (X1_minPT_15_nJets[:,0], X1_wgt, 'Jet minPT 15 Multiplicity NNLO (MiNNLO) \n dileptonic decays')]


bins = np.linspace(-0.5, 10.5, 12)

DCTR.plot_ratio_cms_2(args_multiplicity, bins = bins, y_scale = 'log', ratio_ylim=[0.7, 1.3], part_label='Jet', arg_label='multiplicity', unit='', inv_unit='')


# jet multiplicity

args_multiplicity = [(X0_minPT_30_nJets[:,0], X0_wgt, 'Jet minPT 30 Multiplicity NLO (hvq) \n dileptonic decays'),
                     (X1_minPT_30_nJets[:,0], X1_wgt, 'Jet minPT 30 Multiplicity NNLO (MiNNLO) \n dileptonic decays')]


DCTR.plot_ratio_cms_2(args_multiplicity, bins = bins, y_scale = 'log', ratio_ylim=[0.7, 1.3], part_label='Jet', arg_label='multiplicity', unit='', inv_unit='')


# jet multiplicity

args_multiplicity = [(X0_minPT_50_nJets[:,0], X0_wgt, 'Jet minPT 50 Multiplicity NLO (hvq) \n dileptonic decays'),
                     (X1_minPT_50_nJets[:,0], X1_wgt, 'Jet minPT 50 Multiplicity NNLO (MiNNLO) \n dileptonic decays')]


DCTR.plot_ratio_cms_2(args_multiplicity, bins = bins, y_scale = 'log', ratio_ylim=[0.7, 1.3], part_label='Jet', arg_label='multiplicity', unit='', inv_unit='')



### pT(tt)

In [ ]:
# jet cut_off should have no impact on parton pT

# hvq
args = [(X0_minPT_15, X0_wgt, 'showered NLO (hvq) Jet minPT 15 \n dileptonic decays'),
        (X0_minPT_30, X0_wgt, 'showered NLO (hvq) Jet minPT 30 \n dileptonic decays'),
        (X0_minPT_50, X0_wgt, 'showered NLO (hvq) Jet minPT 50 \n dileptonic decays')]

log_bins = np.logspace(np.log10(1), np.log10(1000), 15)


DCTR.plot_ratio_cms(args, bins = log_bins, y_scale = 'log', ratio_ylim=[0.8, 1.2])


# MiNNLO
args = [(X1_minPT_15, X1_wgt, 'showered NNLO (MiNNLO) Jet minPT 15 \n dileptonic decays'),
        (X1_minPT_30, X1_wgt, 'showered NNLO (MiNNLO) Jet minPT 30 \n dileptonic decays'),
        (X1_minPT_50, X1_wgt, 'showered NNLO (MiNNLO) Jet minPT 50 \n dileptonic decays')]

log_bins = np.logspace(np.log10(1), np.log10(1000), 15)


DCTR.plot_ratio_cms(args, bins = log_bins, y_scale = 'log', ratio_ylim=[0.8, 1.2])



# compare against old shower (HadronLevel:all = on)

In [ ]:
num_events_old = 100000
# dileptonic hvq has all 1M events in one LHE
X0_old = np.load(f'{data_dir}/POWHEG_hvq/dileptonic/converted_lhe_hvq_dileptonic.npy')[:num_events_old]

# nrm data and save to disk (only needed to run once)
'''
nrm_array_dir = f'{data_dir}/POWHEG_hvq/showered/'
nrm_array = np.load(f'{nrm_array_dir}/norm_array_lhe_01.npy')
X0_old_nrm, _ = DCTR.normalize_data(X0_old.copy(), nrm_array)
np.save(f'{data_dir}/POWHEG_hvq/dileptonic/normed_lhe_hvq_dileptonic.npy', X0_old_nrm)
'''

X0_old_nrm = np.array(np.load(f'{data_dir}/POWHEG_hvq/dileptonic/normed_lhe_hvq_dileptonic.npy')[:num_events_old])

X0_old_nJets = np.array(np.load(f'{data_dir}/POWHEG_hvq/dileptonic/jet_multiplicity_hvq_dileptonic.npy')[:num_events_old])

X0_old_wgt = X0_old_nJets[:, 1]

X0_old_jet = np.array(np.load(f'{data_dir}/POWHEG_hvq/dileptonic/jet_4vectors_hvq_dileptonic.npy')[:num_events_old])

print(X0_old.shape)
print(X0_old_nrm.shape)
print(X0_old_nJets.shape)
print(X0_old_jet.shape)


In [ ]:
# jet multiplicity
args_multiplicity = [(X0_minPT_30_nJets[:,0], X0_wgt, 'Jet minPT 30 Multiplicity NLO (hvq) NEW'),
                     (X0_old_nJets[:,0], X0_old_wgt, 'Jet minPT 30 Multiplicity NLO (hvq) OLD \n With HadronLevel:all = on')]


bins = np.linspace(-0.5, 12.5, 14)

DCTR.plot_ratio_cms_2(args_multiplicity, bins = bins, y_scale = 'log', ratio_ylim=[0.7, 1.3], part_label='Jet', arg_label='multiplicity', unit='', inv_unit='')


In [ ]:
# pT
args = [(X0_minPT_30, X0_wgt, 'Jet minPT 30 NLO (hvq) NEW'),
        (X0_old, X0_old_wgt, 'Jet minPT 30 NLO (hvq) OLD \n With HadronLevel:all = on')]



bins = np.logspace(np.log10(1), np.log10(800), 15)

DCTR.plot_ratio_cms_2(args, bins = bins, y_scale = 'log', ratio_ylim=[0.7, 1.3])


# DCTR reweighting


In [ ]:
# calculate weights from DCTR trained on showered events
model = '../20240521_showered_new/train_20240523_regular_epochs.tf' # path to previously trained model (in .tf format (folder)) for showered events

dctr_rwgt = []
# calculate rwgt
with tf.device('CPU'):
    dctr_rwgt_minPT_15 = DCTR.get_rwgt([model], X0_minPT_15_nrm) # .tf models also include network architecture. get_rwgt() sets up the network for the (list of) models, then calls predict_weights() like in DCTR_notebook_OLD_14-to-13TeV notebook

wgts_plot = [(dctr_rwgt_minPT_15[0], r'DCTR NLO $\to$ NNLO reweights minPT 15')]

DCTR.plot_weights(wgts_plot, start = 0.1, stop = 10)

# apply orginal generator weights to rwgt
dctr_rwgt_minPT_15 = np.multiply(dctr_rwgt_minPT_15[0], X0_wgt)



In [ ]:
# calculate weights from DCTR trained on showered events
model = '../20240521_showered_new/train_20240523_regular_epochs.tf' # path to previously trained model (in .tf format (folder)) for showered events

dctr_rwgt = []
# calculate rwgt
with tf.device('CPU'):
    dctr_rwgt_minPT_30 = DCTR.get_rwgt([model], X0_minPT_30_nrm) # .tf models also include network architecture. get_rwgt() sets up the network for the (list of) models, then calls predict_weights() like in DCTR_notebook_OLD_14-to-13TeV notebook

wgts_plot = [(dctr_rwgt_minPT_30[0], r'DCTR NLO $\to$ NNLO reweights minPT 30')]

DCTR.plot_weights(wgts_plot, start = 0.1, stop = 10)

# apply orginal generator weights to rwgt
dctr_rwgt_minPT_30 = np.multiply(dctr_rwgt_minPT_30[0], X0_wgt)



In [ ]:
# calculate weights from DCTR trained on showered events
model = '../20240521_showered_new/train_20240523_regular_epochs.tf' # path to previously trained model (in .tf format (folder)) for showered events

dctr_rwgt = []
# calculate rwgt
with tf.device('CPU'):
    dctr_rwgt_minPT_50 = DCTR.get_rwgt([model], X0_minPT_50_nrm) # .tf models also include network architecture. get_rwgt() sets up the network for the (list of) models, then calls predict_weights() like in DCTR_notebook_OLD_14-to-13TeV notebook

wgts_plot = [(dctr_rwgt_minPT_50[0], r'DCTR NLO $\to$ NNLO reweights minPT 50')]

DCTR.plot_weights(wgts_plot, start = 0.1, stop = 10)

# apply orginal generator weights to rwgt
dctr_rwgt_minPT_50 = np.multiply(dctr_rwgt_minPT_50[0], X0_wgt)



In [ ]:
# jet multiplicity

args_multiplicity = [(X1_minPT_15_nJets[:,0], X1_wgt, 'Jet minPT 15 Multiplicity NNLO (MiNNLO) \n dileptonic decays'),
                     (X0_minPT_15_nJets[:,0], X0_wgt, 'Jet minPT 15 Multiplicity NLO (hvq) \n dileptonic decays'),
                     (X0_minPT_15_nJets[:,0], dctr_rwgt_minPT_15, r'DCTR reweighted NLO $\to$ NNLO')]

bins = np.linspace(-0.5, 10.5, 12)
ratio_ylim=[0.7, 1.3]


DCTR.plot_ratio_cms(args_multiplicity, bins = bins, y_scale = 'log', ratio_ylim=ratio_ylim, part_label='Jet', arg_label='multiplicity', unit='', inv_unit='')



args_multiplicity = [(X1_minPT_30_nJets[:,0], X1_wgt, 'Jet minPT 30 Multiplicity NNLO (MiNNLO) \n dileptonic decays'),
                     (X0_minPT_30_nJets[:,0], X0_wgt, 'Jet minPT 30 Multiplicity NLO (hvq) \n dileptonic decays'),
                     (X0_minPT_30_nJets[:,0], dctr_rwgt_minPT_30, r'DCTR reweighted NLO $\to$ NNLO')]

DCTR.plot_ratio_cms(args_multiplicity, bins = bins, y_scale = 'log', ratio_ylim=ratio_ylim, part_label='Jet', arg_label='multiplicity', unit='', inv_unit='')



args_multiplicity = [(X1_minPT_50_nJets[:,0], X1_wgt, 'Jet minPT 50 Multiplicity NNLO (MiNNLO) \n dileptonic decays'),
                     (X0_minPT_50_nJets[:,0], X0_wgt, 'Jet minPT 50 Multiplicity NLO (hvq) \n dileptonic decays'),
                     (X0_minPT_50_nJets[:,0], dctr_rwgt_minPT_50, r'DCTR reweighted NLO $\to$ NNLO')]

DCTR.plot_ratio_cms(args_multiplicity, bins = bins, y_scale = 'log', ratio_ylim=ratio_ylim, part_label='Jet', arg_label='multiplicity', unit='', inv_unit='')



In [ ]:
# pT

# jet multiplicity

args = [(X1_minPT_15, X1_wgt, 'Jet minPT 15 NNLO (MiNNLO) \n dileptonic decays'),
        (X0_minPT_15, X0_wgt, 'Jet minPT 15 NLO (hvq) \n dileptonic decays'),
        (X0_minPT_15, dctr_rwgt_minPT_15, r'DCTR reweighted NLO $\to$ NNLO')]


bins = np.logspace(np.log10(1), np.log10(800), 15)

DCTR.plot_ratio_cms(args, bins = bins, y_scale = 'log', ratio_ylim=[0.7, 1.3])


# jet multiplicity

args = [(X1_minPT_30, X1_wgt, 'Jet minPT 30 NNLO (MiNNLO) \n dileptonic decays'),
        (X0_minPT_30, X0_wgt, 'Jet minPT 30 NLO (hvq) \n dileptonic decays'),
        (X0_minPT_30, dctr_rwgt_minPT_30, r'DCTR reweighted NLO $\to$ NNLO')]


DCTR.plot_ratio_cms(args, bins = bins, y_scale = 'log', ratio_ylim=[0.7, 1.3])


# jet multiplicity


args = [(X1_minPT_50, X1_wgt, 'Jet minPT 50 NNLO (MiNNLO) \n dileptonic decays'),
        (X0_minPT_50, X0_wgt, 'Jet minPT 50 NLO (hvq) \n dileptonic decays'),
        (X0_minPT_50, dctr_rwgt_minPT_50, r'DCTR reweighted NLO $\to$ NNLO')]


DCTR.plot_ratio_cms(args, bins = bins, y_scale = 'log', ratio_ylim=[0.7, 1.3])



# jet p_t min pt 15

In [ ]:
log_bins = np.logspace(np.log10(30), np.log10(800), 15)

args_jet_0 = [(X1_minPT_15_jet[:, 0, 0], X1_wgt, 'NNLO (MiNNLO)'),
              (X0_minPT_15_jet[:, 0, 0], X0_wgt, 'NLO (hvq)'),
              (X0_minPT_15_jet[:, 0, 0], dctr_rwgt_minPT_15, 'dctr reweighted')]

DCTR.plot_ratio_cms(args_jet_0, bins = log_bins, y_scale = 'log', part_label='First Jet', ratio_ylim=(0.7, 1.3))

log_bins = np.logspace(np.log10(30), np.log10(600), 15)
args_jet_1 = [(X1_minPT_15_jet[:, 1, 0], X1_wgt, 'NNLO (MiNNLO)'),
              (X0_minPT_15_jet[:, 1, 0], X0_wgt, 'NLO (hvq)'),
              (X0_minPT_15_jet[:, 1, 0], dctr_rwgt_minPT_15, 'dctr reweighted')]

DCTR.plot_ratio_cms(args_jet_1, bins = log_bins, y_scale = 'log', part_label='2. Jet', ratio_ylim=(0.7, 1.3))

log_bins = np.logspace(np.log10(30), np.log10(500), 15)
args_jet_2 = [(X1_minPT_15_jet[:, 2, 0], X1_wgt, 'NNLO (MiNNLO)'),
              (X0_minPT_15_jet[:, 2, 0], X0_wgt, 'NLO (hvq)'),
              (X0_minPT_15_jet[:, 2, 0], dctr_rwgt_minPT_15, 'dctr reweighted')]

DCTR.plot_ratio_cms(args_jet_2, bins = log_bins, y_scale = 'log', part_label='3. Jet', ratio_ylim=(0.7, 1.3))

log_bins = np.logspace(np.log10(30), np.log10(200), 15)
args_jet_3 = [(X1_minPT_15_jet[:, 3, 0], X1_wgt, 'NNLO (MiNNLO)'),
              (X0_minPT_15_jet[:, 3, 0], X0_wgt, 'NLO (hvq)'),
              (X0_minPT_15_jet[:, 3, 0], dctr_rwgt_minPT_15, 'dctr reweighted')]

DCTR.plot_ratio_cms(args_jet_3, bins = log_bins, y_scale = 'log', part_label='4. Jet', ratio_ylim=(0.7, 1.3))



In [ ]:
# sum of jet pt per event

X0_minPT_15_jet_pt_sums = []
for event in X0_minPT_15_jet:
    X0_minPT_15_jet_pt_sums.append(np.sum(event[:,0]))

X1_minPT_15_jet_pt_sums = []
for event in X1_minPT_15_jet:
    X1_minPT_15_jet_pt_sums.append(np.sum(event[:,0]))

    
print(f'{np.shape(X0_minPT_15_jet_pt_sums) = }')
print(f'{np.shape(X1_minPT_15_jet_pt_sums) = }')
log_bins = np.logspace(np.log10(10), np.log10(1000), 15)

args_jet_sum = [(np.array(X1_minPT_15_jet_pt_sums), X1_wgt, 'NNLO (MiNNLO)'),
                (np.array(X0_minPT_15_jet_pt_sums), X0_wgt, 'NLO (hvq)'),
                (np.array(X0_minPT_15_jet_pt_sums), dctr_rwgt_minPT_15, 'dctr reweighted')]

#log_bins = np.logspace(np.log10(30), np.log10(1000), 15)
lin_bins = np.linspace(15, 800, 15)

DCTR.plot_ratio_cms(args_jet_sum, bins = lin_bins, y_scale = 'log', part_label='Sum of Jets', ratio_ylim=(0.9, 1.1))




# jet p_t min pt 30

In [ ]:
# jet p_t min pt 30
log_bins = np.logspace(np.log10(30), np.log10(800), 15)

args_jet_0 = [(X1_minPT_50_jet[:, 0, 0], X1_wgt, 'NNLO (MiNNLO)'),
              (X0_minPT_50_jet[:, 0, 0], X0_wgt, 'NLO (hvq)'),
              (X0_minPT_50_jet[:, 0, 0], dctr_rwgt_minPT_50, 'dctr reweighted')]

DCTR.plot_ratio_cms(args_jet_0, bins = log_bins, y_scale = 'log', part_label='First Jet', ratio_ylim=(0.7, 1.3))

log_bins = np.logspace(np.log10(30), np.log10(600), 15)
args_jet_1 = [(X1_minPT_50_jet[:, 1, 0], X1_wgt, 'NNLO (MiNNLO)'),
              (X0_minPT_50_jet[:, 1, 0], X0_wgt, 'NLO (hvq)'),
              (X0_minPT_50_jet[:, 1, 0], dctr_rwgt_minPT_50, 'dctr reweighted')]

DCTR.plot_ratio_cms(args_jet_1, bins = log_bins, y_scale = 'log', part_label='2. Jet', ratio_ylim=(0.7, 1.3))

log_bins = np.logspace(np.log10(30), np.log10(500), 15)
args_jet_2 = [(X1_minPT_50_jet[:, 2, 0], X1_wgt, 'NNLO (MiNNLO)'),
              (X0_minPT_50_jet[:, 2, 0], X0_wgt, 'NLO (hvq)'),
              (X0_minPT_50_jet[:, 2, 0], dctr_rwgt_minPT_50, 'dctr reweighted')]

DCTR.plot_ratio_cms(args_jet_2, bins = log_bins, y_scale = 'log', part_label='3. Jet', ratio_ylim=(0.7, 1.3))

log_bins = np.logspace(np.log10(30), np.log10(200), 15)
args_jet_3 = [(X1_minPT_50_jet[:, 3, 0], X1_wgt, 'NNLO (MiNNLO)'),
              (X0_minPT_50_jet[:, 3, 0], X0_wgt, 'NLO (hvq)'),
              (X0_minPT_50_jet[:, 3, 0], dctr_rwgt_minPT_50, 'dctr reweighted')]

DCTR.plot_ratio_cms(args_jet_3, bins = log_bins, y_scale = 'log', part_label='4. Jet', ratio_ylim=(0.7, 1.3))


# sum of jet pt per event

X0_minPT_50_jet_pt_sums = []
for event in X0_minPT_50_jet:
    X0_minPT_50_jet_pt_sums.append(np.sum(event[:,0]))

X1_minPT_50_jet_pt_sums = []
for event in X1_minPT_50_jet:
    X1_minPT_50_jet_pt_sums.append(np.sum(event[:,0]))

    
print(f'{np.shape(X0_minPT_50_jet_pt_sums) = }')
print(f'{np.shape(X1_minPT_50_jet_pt_sums) = }')
log_bins = np.logspace(np.log10(10), np.log10(1000), 15)

args_jet_sum = [(np.array(X1_minPT_50_jet_pt_sums), X1_wgt, 'NNLO (MiNNLO)'),
                (np.array(X0_minPT_50_jet_pt_sums), X0_wgt, 'NLO (hvq)'),
                (np.array(X0_minPT_50_jet_pt_sums), dctr_rwgt_minPT_50, 'dctr reweighted')]

#log_bins = np.logspace(np.log10(30), np.log10(1000), 15)
lin_bins = np.linspace(30, 800, 15)

DCTR.plot_ratio_cms(args_jet_sum, bins = lin_bins, y_scale = 'log', part_label='Sum of Jets', ratio_ylim=(0.9, 1.1))




# jet p_t min pt 50

In [ ]:
# jet p_t min pt 50
log_bins = np.logspace(np.log10(30), np.log10(800), 15)

args_jet_0 = [(X1_minPT_30_jet[:, 0, 0], X1_wgt, 'NNLO (MiNNLO)'),
              (X0_minPT_30_jet[:, 0, 0], X0_wgt, 'NLO (hvq)'),
              (X0_minPT_30_jet[:, 0, 0], dctr_rwgt_minPT_30, 'dctr reweighted')]

DCTR.plot_ratio_cms(args_jet_0, bins = log_bins, y_scale = 'log', part_label='First Jet', ratio_ylim=(0.7, 1.3))

log_bins = np.logspace(np.log10(30), np.log10(600), 15)
args_jet_1 = [(X1_minPT_30_jet[:, 1, 0], X1_wgt, 'NNLO (MiNNLO)'),
              (X0_minPT_30_jet[:, 1, 0], X0_wgt, 'NLO (hvq)'),
              (X0_minPT_30_jet[:, 1, 0], dctr_rwgt_minPT_30, 'dctr reweighted')]

DCTR.plot_ratio_cms(args_jet_1, bins = log_bins, y_scale = 'log', part_label='2. Jet', ratio_ylim=(0.7, 1.3))

log_bins = np.logspace(np.log10(30), np.log10(500), 15)
args_jet_2 = [(X1_minPT_30_jet[:, 2, 0], X1_wgt, 'NNLO (MiNNLO)'),
              (X0_minPT_30_jet[:, 2, 0], X0_wgt, 'NLO (hvq)'),
              (X0_minPT_30_jet[:, 2, 0], dctr_rwgt_minPT_30, 'dctr reweighted')]

DCTR.plot_ratio_cms(args_jet_2, bins = log_bins, y_scale = 'log', part_label='3. Jet', ratio_ylim=(0.7, 1.3))

log_bins = np.logspace(np.log10(30), np.log10(200), 15)
args_jet_3 = [(X1_minPT_30_jet[:, 3, 0], X1_wgt, 'NNLO (MiNNLO)'),
              (X0_minPT_30_jet[:, 3, 0], X0_wgt, 'NLO (hvq)'),
              (X0_minPT_30_jet[:, 3, 0], dctr_rwgt_minPT_30, 'dctr reweighted')]

DCTR.plot_ratio_cms(args_jet_3, bins = log_bins, y_scale = 'log', part_label='4. Jet', ratio_ylim=(0.7, 1.3))


# sum of jet pt per event

X0_minPT_30_jet_pt_sums = []
for event in X0_minPT_30_jet:
    X0_minPT_30_jet_pt_sums.append(np.sum(event[:,0]))

X1_minPT_30_jet_pt_sums = []
for event in X1_minPT_30_jet:
    X1_minPT_30_jet_pt_sums.append(np.sum(event[:,0]))

    
print(f'{np.shape(X0_minPT_30_jet_pt_sums) = }')
print(f'{np.shape(X1_minPT_30_jet_pt_sums) = }')
log_bins = np.logspace(np.log10(10), np.log10(1000), 15)

args_jet_sum = [(np.array(X1_minPT_30_jet_pt_sums), X1_wgt, 'NNLO (MiNNLO)'),
                (np.array(X0_minPT_30_jet_pt_sums), X0_wgt, 'NLO (hvq)'),
                (np.array(X0_minPT_30_jet_pt_sums), dctr_rwgt_minPT_30, 'dctr reweighted')]

#log_bins = np.logspace(np.log10(30), np.log10(1000), 15)
lin_bins = np.linspace(30, 800, 15)

DCTR.plot_ratio_cms(args_jet_sum, bins = lin_bins, y_scale = 'log', part_label='Sum of Jets', ratio_ylim=(0.9, 1.1))


